# WEC AC Injection

### Imports

In [2]:
import sys
sys.path.append('../wec-grid-code') # 
import wec_grid_class as wg
import sqlite3
import pandas as pd

In [3]:
# Read in case file 
case14 = r"../input_files/case14.raw"

### run for DC 

In [ ]:
# initalize WEC GRID Object
pf = wg.Wec_grid(case14,"DC",3)

### run for AC 

In [5]:
# initalize WEC GRID Object
pf = wg.Wec_grid(case14,"fnsl",3)


 Reading IC, SBASE, REV, XFRRAT, NXFRAT, BASFRQ...

 Converting RAW data from PSS(R)E 33 to PSS(R)E 34 format
 Starting processing of Case identification data records
 Finished processing of 3 records for Case identification data
 Starting processing of Bus data records
 Finished processing of 14 records for Bus data
 Starting processing of Load data records
 Finished processing of 11 records for Load data
 Starting processing of Fixed shunt data records
 Finished processing of 1 records for Fixed shunt data
 Starting processing of Generator data records
 Finished processing of 5 records for Generator data
 Starting processing of Branch data records
 Finished processing of 17 records for Branch data
 Finished processing of 0 records for System switching device data
 Starting processing of Transformer data records
 Finished processing of 12 records for 3 Transformer data
 Starting processing of Area data records
 Finished processing of 0 records for Area data
 Starting processing of Tw

### Run WEC Model via matlab (takes a minute)

In [ ]:
#pf.run_WEC_Sim()

In [ ]:
pf.dataframe

### Pull WEC injection values via SQL database

In [ ]:
con = sqlite3.connect("../input_files/r2g_database.db")
injection = pd.read_sql_query("SELECT * from WEC_output", con)
print(injection)

In [ ]:
for i in range(len(injection)):
    print("Time: {}".format(injection.iloc[i].time))
    print("P setpoint: {}".format(injection.pg.iloc[i]))
    if pf.solver == 'AC':
        pf.ac_injection(pf.wecBus_num, injection.pg.iloc[i], injection.vs.iloc[i],
                        pf.solver, injection.iloc[i].time)
    if pf.solver == 'DC':
        pf.dc_injection(pf.wecBus_num, injection.pg.iloc[i], pf.solver, injection.iloc[i].time)
    print("========================")

In [ ]:
injection

In [ ]:
pf.plotSwingBus()

In [ ]:
pf.plotWecBus()